<a href="https://colab.research.google.com/github/skaurl/NH-bigdata/blob/main/classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-lxtx3zxu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-lxtx3zxu
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=0c4e500de9145195acc30f01b6706e8dbedc53bfce4e05bdec118a5ad266d2f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-bj9qr0xl/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


# Preprocessing

In [6]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

news_dataset_path = r"/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/open/news_train.csv"

news_dataset = pd.read_csv(news_dataset_path)

news_dataset = news_dataset[news_dataset["content"]!="데이터에서 찾은 AI 속보"]
news_dataset = news_dataset[news_dataset["content"]!="씨에스윈드, 193억 원 규모 윈드타워 공급계약 체결"]
news_dataset = news_dataset[news_dataset["content"]!="씨에스윈드, 1분기 영업이익 158억 원전년비 16.1%"]

news_dataset["content"] = news_dataset["title"] + " " + news_dataset["content"]

del news_dataset["n_id"]
del news_dataset["date"]
del news_dataset["title"]
del news_dataset["ord"]

news_dataset = news_dataset.drop_duplicates()

news_dataset["content"] = news_dataset["content"].apply(lambda x : ' '.join(x.strip().split()))

train, test = train_test_split(news_dataset, test_size=0.1, random_state=42)

train.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_train_train.txt", sep='\t', index=False)
test.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_train_test.txt", sep='\t', index=False)

In [7]:
dataset_train = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_train_train.txt", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_train_test.txt", field_indices=[0,1], num_discard_samples=1)

In [8]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
max_len = 128
batch_size = 64
warmup_ratio = 0.01
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# Classification

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [15]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [16]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [17]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [18]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [19]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [20]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7124210596084595 train acc 0.515625
epoch 1 batch id 201 loss 0.03198886662721634 train acc 0.9144123134328358
epoch 1 batch id 401 loss 0.028238805010914803 train acc 0.9443188902743143
epoch 1 batch id 601 loss 0.10605821013450623 train acc 0.9565307820299501
epoch 1 batch id 801 loss 0.08118673413991928 train acc 0.9635026529338327

epoch 1 train acc 0.968088522643402


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9900056306306306


epoch 2 batch id 1 loss 0.053626712411642075 train acc 0.984375
epoch 2 batch id 201 loss 0.004112955182790756 train acc 0.9892723880597015
epoch 2 batch id 401 loss 0.0024008832406252623 train acc 0.9909990648379052
epoch 2 batch id 601 loss 0.07933811098337173 train acc 0.9919145174708819
epoch 2 batch id 801 loss 0.10948361456394196 train acc 0.9926068976279651

epoch 2 train acc 0.9932923771313942


...


epoch 3 test acc 0.9936224489795918


epoch 4 batch id 1 loss 0.005763388238847256 train acc 1.0
epoch 4 batch id 201 loss 0.0001191279516206123 train acc 0.9991449004975125
epoch 4 batch id 401 loss 0.00019714119844138622 train acc 0.9992206982543641
epoch 4 batch id 601 loss 0.004905913025140762 train acc 0.9991940515806988
epoch 4 batch id 801 loss 5.113134466228075e-05 train acc 0.9993367665418227

epoch 4 train acc 0.9993731193580743



epoch 4 test acc 0.9934385916528773


epoch 5 batch id 1 loss 0.00011151501530548558 train acc 1.0
epoch 5 batch id 201 loss 0.00012967966904398054 train acc 0.9997667910447762
epoch 5 batch id 401 loss 5.405384217738174e-05 train acc 0.9996493142144638
epoch 5 batch id 601 loss 5.587272607954219e-05 train acc 0.9996620216306157
epoch 5 batch id 801 loss 6.129868415882811e-05 train acc 0.9997464107365793

epoch 5 train acc 0.9997649197592778



epoch 5 test acc 0.9944670435741865


In [21]:
torch.save(model.state_dict(), "/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/model/model_state_dict.pt")

# 채점

데이터 불러오기

In [22]:
import pandas as pd

#test = pd.read_csv('/content/news_test.csv', encoding = 'utf-8-sig')

news_dataset_path = r"/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/open/news_test.csv"

news_dataset = pd.read_csv(news_dataset_path)

시간 측정 시작

In [23]:
import time
start = time.time()

Library 불러오기

pos_Tagger, Tokenizer, pretraind_embedding, Model 불러오기

In [24]:
model.load_state_dict(torch.load("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/model/model_state_dict.pt"))

<All keys matched successfully>

형태소 분석 + 전처리

In [25]:
news_dataset["content"] = news_dataset["title"] + " " + news_dataset["content"]

del news_dataset["n_id"]
del news_dataset["date"]
del news_dataset["title"]
del news_dataset["ord"]

news_dataset["content"] = news_dataset["content"].apply(lambda x : ' '.join(x.strip().split()))

news_dataset["info"] = 0

news_dataset[["content","info"]].to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_test.txt", sep='\t', index=False)

dataset_test = nlp.data.TSVDataset("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/data/news_test.txt", field_indices=[0,1], num_discard_samples=1)

data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=5)

예측

In [26]:
model.eval()

Predict = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(out,1)
    Predict.append(int(predict[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [28]:
news_dataset["info"] = Predict

print(news_dataset.groupby("info").size())

news_dataset = news_dataset[["id","info"]]

info
0    87621
1    54944
dtype: int64


In [29]:
print(time.time() - start)

3089.563986301422


제출

In [31]:
news_dataset.to_csv("/gdrive/My Drive/한양대학교/NH투자증권 빅데이터 경진대회/result/sample_submission.csv", index=False)